In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('municipality_bus_utilization.csv')
df.head()

,timestamp,municipality_id,usage,total_capacity
0,2017-06-04 07:59:42,9,454,1332
1,2017-06-04 07:59:42,8,556,2947
2,2017-06-04 07:59:42,4,1090,3893
3,2017-06-04 07:59:42,0,204,2813
4,2017-06-04 07:59:42,7,718,2019


In [3]:
df.tail()

,timestamp,municipality_id,usage,total_capacity
13065,2017-08-19 16:30:35,2,548,697
13066,2017-08-19 16:30:35,8,1193,2947
13067,2017-08-19 16:30:35,7,1354,2019
13068,2017-08-19 16:30:35,6,1680,3113
13069,2017-08-19 16:30:35,3,1189,1930


In [4]:
df.shape

(13070, 4)

In [5]:
df['free_to_use'] = df['total_capacity'] - df['usage']
df

,timestamp,municipality_id,usage,total_capacity,free_to_use
0,2017-06-04 07:59:42,9,454,1332,878
1,2017-06-04 07:59:42,8,556,2947,2391
2,2017-06-04 07:59:42,4,1090,3893,2803
3,2017-06-04 07:59:42,0,204,2813,2609
4,2017-06-04 07:59:42,7,718,2019,1301
...,...,...,...,...,...
13065,2017-08-19 16:30:35,2,548,697,149
13066,2017-08-19 16:30:35,8,1193,2947,1754
13067,2017-08-19 16:30:35,7,1354,2019,665
13068,2017-08-19 16:30:35,6,1680,3113,1433


In [6]:
(df['free_to_use'] <= 0).sum()

241

In [7]:
(df['free_to_use'] < 0 ).sum()

229

In [8]:
df[df['free_to_use'] < 0 ]

,timestamp,municipality_id,usage,total_capacity,free_to_use
7618,2017-07-17 12:04:02,1,399,397,-2
7621,2017-07-17 12:31:03,1,401,397,-4
7635,2017-07-17 13:04:03,1,399,397,-2
7648,2017-07-17 13:31:03,1,403,397,-6
7659,2017-07-17 14:04:03,1,403,397,-6
...,...,...,...,...,...
13010,2017-08-19 14:03:31,1,408,397,-11
13026,2017-08-19 14:30:33,1,410,397,-13
13037,2017-08-19 15:03:34,1,408,397,-11
13048,2017-08-19 15:29:33,1,411,397,-14


In [9]:
df[df['free_to_use'] < 0 ]['municipality_id'].value_counts()

1    229
Name: municipality_id, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13070 entries, 0 to 13069
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        13070 non-null  object
 1   municipality_id  13070 non-null  int64 
 2   usage            13070 non-null  int64 
 3   total_capacity   13070 non-null  int64 
 4   free_to_use      13070 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 510.7+ KB


In [ ]:
df = df.sort_values('timestamp' , ascending=True)
df

In [11]:
df['timestamp'].value_counts(dropna=False)

2017-06-04 07:59:42    10
2017-07-22 15:31:37    10
2017-07-23 15:31:42    10
2017-07-23 15:04:43    10
2017-07-23 14:30:43    10
                       ..
2017-08-19 16:30:35    10
2017-06-16 14:34:14     9
2017-08-11 08:25:43     9
2017-06-16 14:27:13     1
2017-08-11 08:19:39     1
Name: timestamp, Length: 1309, dtype: int64

In [12]:
df[df['timestamp'] == '2017-06-16 14:27:13']

,timestamp,municipality_id,usage,total_capacity,free_to_use
2290,2017-06-16 14:27:13,5,189,587,398


In [13]:
df[df['timestamp'] == '2017-06-16 14:34:14']

,timestamp,municipality_id,usage,total_capacity,free_to_use
2291,2017-06-16 14:34:14,4,1348,3893,2545
2292,2017-06-16 14:34:14,3,803,1930,1127
2293,2017-06-16 14:34:14,7,1294,2019,725
2294,2017-06-16 14:34:14,1,338,397,59
2295,2017-06-16 14:34:14,9,754,1332,578
2296,2017-06-16 14:34:14,0,497,2813,2316
2297,2017-06-16 14:34:14,2,549,697,148
2298,2017-06-16 14:34:14,6,983,3113,2130
2299,2017-06-16 14:34:14,8,877,2947,2070


In [15]:
df[df['timestamp'] == '2017-08-11 08:19:39']

,timestamp,municipality_id,usage,total_capacity,free_to_use
11480,2017-08-11 08:19:39,3,313,1930,1617


In [16]:
df[df['timestamp'] == '2017-08-11 08:25:43']

,timestamp,municipality_id,usage,total_capacity,free_to_use
11481,2017-08-11 08:25:43,4,782,3893,3111
11482,2017-08-11 08:25:43,8,450,2947,2497
11483,2017-08-11 08:25:43,5,11,587,576
11484,2017-08-11 08:25:43,9,379,1332,953
11485,2017-08-11 08:25:43,7,739,2019,1280
11486,2017-08-11 08:25:43,2,369,697,328
11487,2017-08-11 08:25:43,0,344,2813,2469
11488,2017-08-11 08:25:43,6,808,3113,2305
11489,2017-08-11 08:25:43,1,132,397,265


In [ ]:
# df[df['timestamp'] == '2017-06-16 14:27:13']['timestamp'] = '2017-06-16 14:34:14'

In [17]:
df.loc[df['timestamp'] == '2017-06-16 14:27:13', 'timestamp'] = '2017-06-16 14:34:14'
df.loc[df['timestamp'] == '2017-08-11 08:19:39', 'timestamp'] = '2017-08-11 08:25:43'

In [18]:
df['timestamp'].value_counts(dropna=False)

2017-06-04 07:59:42    10
2017-07-22 15:57:37    10
2017-07-23 16:04:46    10
2017-07-23 15:31:42    10
2017-07-23 15:04:43    10
                       ..
2017-06-30 09:25:57    10
2017-06-30 08:59:52    10
2017-06-30 08:25:53    10
2017-06-30 07:59:55    10
2017-08-19 16:30:35    10
Name: timestamp, Length: 1307, dtype: int64

In [19]:
df['municipality_id'].value_counts(dropna=False)

9    1307
8    1307
4    1307
0    1307
7    1307
5    1307
2    1307
1    1307
6    1307
3    1307
Name: municipality_id, dtype: int64

In [20]:
df['total_capacity'].value_counts(dropna=False)

1332    1307
2947    1307
3893    1307
2813    1307
2019    1307
587     1307
697     1307
397     1307
3113    1307
1930    1307
Name: total_capacity, dtype: int64

In [21]:
df['usage'].value_counts(dropna=False)

411     69
354     62
399     31
410     28
623     27
        ..
2830     1
2783     1
2664     1
2463     1
1680     1
Name: usage, Length: 2346, dtype: int64

In [22]:
df['timestamp'] = df['timestamp'].astype('datetime64')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13070 entries, 0 to 13069
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   timestamp        13070 non-null  datetime64[ns]
 1   municipality_id  13070 non-null  int64         
 2   usage            13070 non-null  int64         
 3   total_capacity   13070 non-null  int64         
 4   free_to_use      13070 non-null  int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 510.7 KB


In [24]:
df.groupby('timestamp').max()

,municipality_id,usage,total_capacity,free_to_use
timestamp,,,,
2017-06-04 07:59:42,9,1090,3893,2803
2017-06-04 08:25:42,9,1476,3893,2566
2017-06-04 08:59:42,9,2016,3893,2481
2017-06-04 09:32:46,9,2511,3893,2384
2017-06-04 09:59:48,9,2811,3893,2328
...,...,...,...,...
2017-08-19 14:30:33,9,3176,3893,1606
2017-08-19 15:03:34,9,3111,3893,1611
2017-08-19 15:29:33,9,2980,3893,1632


In [25]:
df.groupby('timestamp').min()

,municipality_id,usage,total_capacity,free_to_use
timestamp,,,,
2017-06-04 07:59:42,0,70,397,268
2017-06-04 08:25:42,0,73,397,259
2017-06-04 08:59:42,0,89,397,233
2017-06-04 09:32:46,0,116,397,184
2017-06-04 09:59:48,0,159,397,152
...,...,...,...,...
2017-08-19 14:30:33,0,318,397,-13
2017-08-19 15:03:34,0,309,397,-11
2017-08-19 15:29:33,0,283,397,-14


In [ ]:
# df['timestamp'].head(50)

In [ ]:
# df['timestamp'].tail(50)

In [ ]:
# df['timestamp'][0].second

In [26]:
for i in range(df.shape[0]):
    if df['timestamp'][i].minute == 0 :
        print(i)

3400
3401
3402
3403
3404
3405
3406
3407
3408
3409
3420
3421
3422
3423
3424
3425
3426
3427
3428
3429
3440
3441
3442
3443
3444
3445
3446
3447
3448
3449
4450
4451
4452
4453
4454
4455
4456
4457
4458
4459
4470
4471
4472
4473
4474
4475
4476
4477
4478
4479
4490
4491
4492
4493
4494
4495
4496
4497
4498
4499
4510
4511
4512
4513
4514
4515
4516
4517
4518
4519
4530
4531
4532
4533
4534
4535
4536
4537
4538
4539
4550
4551
4552
4553
4554
4555
4556
4557
4558
4559
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4730
4731
4732
4733
4734
4735
4736
4737
4738
4739
4750
4751
4752
4753
4754
4755
4756
4757
4758
4759
4770
4771
4772
4773
4774
4775
4776
4777
4778
4779
4790
4791
4792
4793
4794
4795
4796
4797
4798
4799
4810
4811
4812
4813
4814
4815
4816
4817
4818
4819


In [27]:
df.iloc[3400,:]

timestamp          2017-06-24 16:00:04
municipality_id                      1
usage                              272
total_capacity                     397
free_to_use                        125
Name: 3400, dtype: object

In [28]:
df.iloc[13039,:]

timestamp          2017-08-19 15:03:34
municipality_id                      6
usage                             1829
total_capacity                    3113
free_to_use                       1284
Name: 13039, dtype: object

In [ ]:
# df.iloc[3340:3370,:].head(80)

In [ ]:
# for i in range(df.shape[0]):
#     if df['timestamp'][i].hour == 7 :
#         print(i)

In [ ]:
# for i in range(df.shape[0]):
#     if df['timestamp'][i].minute > 50 :
#         print(i)

In [29]:
counter7 = 0
counter8 = 0
counter9 = 0
counter10 = 0
counter11 = 0
counter12 = 0
counter13 = 0
counter14 = 0
counter15 = 0
counter16 = 0

for i in range(df.shape[0]):
    if df['timestamp'][i].hour == 7 :
        counter7 += 1
    elif df['timestamp'][i].hour == 8 :
        counter8 += 1
    elif df['timestamp'][i].hour == 9 :
        counter9 += 1
    elif df['timestamp'][i].hour == 10 :
        counter10 += 1
    elif df['timestamp'][i].hour == 11 :
        counter11 += 1
    elif df['timestamp'][i].hour == 12 :
        counter12 += 1
    elif df['timestamp'][i].hour == 13 :
        counter13 += 1
    elif df['timestamp'][i].hour == 14 :
        counter14 += 1
    elif df['timestamp'][i].hour == 15 :
        counter15 += 1
    elif df['timestamp'][i].hour == 16 :
        counter16 += 1
    elif df['timestamp'][i].hour == 17 :
        print('Warning !')

In [30]:
print(f"total {counter7} data at 7 o'clock")
print(f"total {counter8} data at 8 o'clock")
print(f"total {counter9} data at 9 o'clock")
print(f"total {counter10} data at 10 o'clock")
print(f"total {counter11} data at 11 o'clock")
print(f"total {counter12} data at 12 o'clock")
print(f"total {counter13} data at 13 o'clock")
print(f"total {counter14} data at 14 o'clock")
print(f"total {counter15} data at 15 o'clock")
print(f"total {counter16} data at 16 o'clock")

total 260 data at 7 o'clock
total 1450 data at 8 o'clock
total 1490 data at 9 o'clock
total 1410 data at 10 o'clock
total 1460 data at 11 o'clock
total 1420 data at 12 o'clock
total 1520 data at 13 o'clock
total 1450 data at 14 o'clock
total 1450 data at 15 o'clock
total 1160 data at 16 o'clock


In [ ]:
df['timestamp'][0]

In [ ]:
df['month'] = df['timestamp'].apply(lambda x : x.month)
df['day'] = df['timestamp'].apply(lambda x : x.day)
df['hour'] = df['timestamp'].apply(lambda x : x.hour)

In [ ]:
df

In [ ]:
plt.figure(figsize=(16,6))

df.groupby(["month","day"])["usage"].sum().plot();

In [ ]:
plt.figure(figsize=(16,6))

df.groupby(["month","day",'hour'])["usage"].sum().plot();

In [ ]:
# plt.figure(figsize=(12,4))

# df.groupby(["hour"])["usage"].sum().plot();                       # !Meaningless

In [ ]:
df_0 = df[df['municipality_id'] == 0]
df_1 = df[df['municipality_id'] == 1]
df_2 = df[df['municipality_id'] == 2]
df_3 = df[df['municipality_id'] == 3]
df_4 = df[df['municipality_id'] == 4]
df_5 = df[df['municipality_id'] == 5]
df_6 = df[df['municipality_id'] == 6]
df_7 = df[df['municipality_id'] == 7]
df_8 = df[df['municipality_id'] == 8]
df_9 = df[df['municipality_id'] == 9]

# Problems I've found so far

1) Either the total_capacity of the municipality with id number 1 was entered incorrectly or the usage information of 229 data was entered incorrectly.
2) The time of 2 data was entered incorrectly (Solved)
3) dtype of timestap feature was object (Solved)
4) After the 3400.index, it changes to a new hour and the minute becomes 0. Should we leave the data after 3400 as it is and count at the hour it is found or should we include it in the previous hour ?
5) 

In [ ]:
df.groupby('timstamp')

In [ ]:
df